In [4]:
import sqlite3
import pandas as pd

In [5]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [1]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [3]:
!wget https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv

--2022-09-13 09:33:35--  https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 39168 (38K) [text/plain]
Сохранение в: «boston.csv»

boston.csv          100%[===================>]  38,25K  --.-KB/s    за 0,1s    

2022-09-13 09:33:36 (355 KB/s) - «boston.csv» сохранён [39168/39168]



In [6]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [7]:
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [8]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [16]:
data.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [9]:
data.shape

(506, 14)

#### 1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [28]:
pd.read_sql(
    """
SELECT SUM(CRIM IS NULL),
       SUM(ZN IS NULL),
       SUM(INDUS IS NULL),
       SUM(CHAS IS NULL),
       SUM(NOX IS NULL)
FROM boston nt
    """,
    con,
)

,SUM(CRIM IS NULL),SUM(ZN IS NULL),SUM(INDUS IS NULL),SUM(CHAS IS NULL),SUM(NOX IS NULL)
0,0,0,0,0,0


In [39]:
data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX']].isnull().sum()

CRIM     0
ZN       0
INDUS    0
CHAS     0
NOX      0
dtype: int64

#### 2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [29]:
pd.read_sql(
    """
SELECT COUNT(DISTINCT ZN),
	   COUNT(DISTINCT CRIM),
	   COUNT(DISTINCT INDUS),
	   COUNT(DISTINCT CHAS),
	   COUNT(DISTINCT NOX)
FROM boston nt 
    """,
    con,
)

,COUNT(DISTINCT ZN),COUNT(DISTINCT CRIM),COUNT(DISTINCT INDUS),COUNT(DISTINCT CHAS),COUNT(DISTINCT NOX)
0,26,504,76,2,81


In [40]:
data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX']].nunique()

CRIM     504
ZN        26
INDUS     76
CHAS       2
NOX       81
dtype: int64

#### !!!!3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

In [ ]:
data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX']].median()

CRIM     0.25651
ZN       0.00000
INDUS    9.69000
CHAS     0.00000
NOX      0.53800
dtype: float64

#### !!!! 4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

#### 5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [37]:
pd.read_sql(
    """
WITH ranked AS
  (SELECT LSTAT,
          MEDV,
          DENSE_RANK() OVER (
                             ORDER BY LSTAT ASC) AS lstat_rank
   FROM boston nt)
SELECT LSTAT,
       MEDV*1000,
       lstat_rank
FROM ranked
WHERE MEDV =
    (SELECT max(MEDV)
     FROM boston nt)
    """,
    con,
)

,LSTAT,MEDV*1000,lstat_rank
0,1.73,50000.0,1
1,1.92,50000.0,2
2,2.88,50000.0,6
3,2.96,50000.0,8
4,2.97,50000.0,9
5,3.16,50000.0,14
6,3.26,50000.0,15
7,3.32,50000.0,16
8,3.70,50000.0,23
9,3.73,50000.0,24


#### 6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [32]:
pd.read_sql(
    """
SELECT avg(MEDV)*1000 as AvgPriceHouse,
       CHAS
FROM boston nt
WHERE CHAS = 1
UNION ALL
SELECT avg(MEDV)*1000,
       CHAS
FROM boston nt
WHERE CHAS = 0
    """,
    con,
)

,AvgPriceHouse,CHAS
0,28440.000000,1.0
1,22093.842887,0.0


#### 7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

#### 8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)